# flats-in-cracow data wrangling

## Imports

In [217]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from collections import Counter
from IPython.display import display
from sklearn.impute import KNNImputer
from pylab import rcParams
from pathlib import Path

## Setup

In [218]:
# Create directory for images 
Path("img").mkdir(parents=True, exist_ok=True)

# Set default figure size
rcParams['figure.figsize'] = (4, 4)

# Tell pandas how to display floats
pd.options.display.float_format = "{:,.2f}".format

## Goal

I scraped listings of properties for sale in Cracow. We would like to create a model to predict flat prices.

## Data source

Data has been scraped from a website with listings. The data has undergone small transformations along the way. The goal of these transformations was to get the data into a usable state not to check it's validity.

## Data loading

In [219]:
path = '../flats-data/raw_data.csv'

In [220]:
data = pd.read_csv(path, lineterminator='\n')

In [221]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38548 entries, 0 to 38547
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         38448 non-null  object 
 1   City         27407 non-null  object 
 2   District     19997 non-null  object 
 3   Amount       38415 non-null  float64
 4   Currency     38415 non-null  object 
 5   Property     38158 non-null  object 
 6   Seller       38334 non-null  object 
 7   Area         38216 non-null  float64
 8   Rooms        37774 non-null  float64
 9   Bathrooms    24348 non-null  float64
 10  Parking      16129 non-null  object 
 11  Garden       38548 non-null  bool   
 12  Balcony      38548 non-null  bool   
 13  Terrace      38548 non-null  bool   
 14  Floor        38548 non-null  bool   
 15  New          38548 non-null  bool   
 16  Estate       38548 non-null  bool   
 17  Townhouse    38548 non-null  bool   
 18  Apartment    38548 non-null  bool   
 19  Land

I assume that the `Title` column uniquely identifies a listing.

In [222]:
data = data.drop_duplicates(['Title'])

In [223]:
print(data.shape)

(7482, 24)


## Data exploration

We check for missing values that we will have to deal with.

In [224]:
missing = data.isnull().sum(axis=0)
missing.name = 'Missing'
missing = missing.to_frame()
missing = missing[missing['Missing'] > 0]
missing.sort_values('Missing', ascending=False)

,Missing
Parking,4843
District,3194
Bathrooms,3141
City,1334
Description,1206
Rooms,188
Area,95
Seller,66
Property,63
Amount,6


### Check numeric columns

We see that we have 24 columns at our disposal. 
We inspect the numeric columns to see what we are dealing with. 
In the `Amount` column we note there is a property for sale that costs 1PLN, clearly a erroneous value. 
Next we note that the enourmous maximum in the `Amount` column. That is quite a lot of money and could be considered a potential outlier.
The maximum and minimum of the `Area` column also indicate the existance of outliers. These values are clearly too large. The data will need to undergo a filtering process.

In [225]:
data.describe()

,Amount,Area,Rooms,Bathrooms
count,"7,476.00","7,387.00","7,294.00","4,341.00"
mean,"678,943.16",153.28,2.90,1.33
std,"709,720.67","4,217.45",1.32,0.64
min,1.00,1.00,1.00,1.00
25%,"398,153.00",43.00,2.00,1.00
50%,"500,000.00",57.00,3.00,1.00
75%,"720,000.00",81.00,4.00,2.00
max,"22,000,000.00","320,000.00",6.00,4.00


### Check binary columns

We inspect the data to see if binary columns are properly populated and check for imbalances.

In [226]:
binary = data.select_dtypes(bool).columns.to_list()

for col in binary:
    tmp = data[[col, 'Amount']]
    tmp = tmp.fillna('NaN')
    tmp = tmp.groupby(col, as_index=False)
    tmp = tmp.count()
    tmp = tmp.rename(columns={'Amount': 'Count'})
    tmp = tmp.sort_values('Count', ascending=False)
    tmp = tmp.reset_index(drop=True)
    display(tmp)

,Garden,Count
0,False,6015
1,True,1467


,Balcony,Count
0,False,4888
1,True,2594


,Terrace,Count
0,False,6585
1,True,897


,Floor,Count
0,False,4530
1,True,2952


,New,Count
0,False,5000
1,True,2482


,Estate,Count
0,False,6826
1,True,656


,Townhouse,Count
0,False,6810
1,True,672


,Apartment,Count
0,False,6400
1,True,1082


,Land,Count
0,False,5813
1,True,1669


,Studio,Count
0,False,7009
1,True,473


### Check categorical columns

We inspect categorical columns to assert that they contain "valid" values. Most of these columns were generated by a script during the scraping and etl phase of the project.

In [227]:
categorical = data.select_dtypes('object').columns
categorical = categorical.to_list()
omit = ['Title', 'Link', 'Description', 'Date']

for col in categorical:
    if col not in omit:
        tmp = data[['Amount', col]].copy()
        tmp = tmp.fillna('NaN')
        tmp = tmp.groupby(col, as_index=False)
        tmp = tmp.count()
        tmp = tmp.rename(columns={'Amount': 'Count'})
        tmp = tmp.sort_values('Count', ascending=False)
        tmp = tmp.reset_index(drop=True)
        display(tmp)

,City,Count
0,kraków,6148
1,NaN,1334


,District,Count
0,NaN,3194
1,krowodrza,642
2,podgórze,530
3,stare miasto,506
4,nowa huta,372
5,bronowice,303
6,dębniki,285
7,prądnik biały,244
8,grzegórzki,204
9,prądnik czerwony,200


,Currency,Count
0,pln,7476
1,NaN,6


,Property,Count
0,flat,6412
1,house,1007
2,NaN,63


,Seller,Count
0,realtor,6886
1,owner,530
2,NaN,66


,Parking,Count
0,NaN,4843
1,street,1077
2,garage,1004
3,no parking,421
4,covered,137


### Check text columns

We search for keywords in the data.

In [228]:
# text = data[data['Description'].isna() == False].copy()
# text = text['Description'].to_list()
# text = ' '.join(text)
# text = text.split(' ')
# text = [x for x in text if x.isalpha()]
# text = [x for x in text if len(x) > 3]

In [229]:
# for i in range(5, len(text)-5):
#     if 'piętro' in text[i]:    
#         s = text[i-5:i+5]
#         s = ' '.join(s)
#         print(s)

## Data cleaning

We assume that if we know the district, the `City` is `kraków`.

In [230]:
mask = (data['City'].isna() == True) & (data['District'].isna() == False)
data.loc[mask, 'City'] = 'kraków'

We extract more `Parking` information from the property description.

In [231]:
def extract_parking(x):
    if ('garaż' in x or 'garaz' in x or 'parking' in x) and 'podziemny' in x:
        return 'covered'
    elif ('garaż' in x or 'garaz' in x) and 'podziemny' not in x:
        return 'garage'
    elif 'parking' in x and 'podziemny' not in x:
        return 'street'
    else:
        return 'no parking'

In [232]:
mask = (data['Parking'].isna() == True) & (data['Description'].isna() == False)
data.loc[mask, ['Parking', 'Description']] = data.loc[mask, 'Description'].apply(extract_parking)

In [233]:
mask = data['Parking'].isna() == True
data.loc[mask, 'Parking'] = 'no parking'

We confirm that we have dealt with all the `NaN`s in the `Parking` column.

In [234]:
print(data['Parking'].isna().sum())

0


### Filtering

Next we filter the data according to these rules:

In [235]:
data = data[data['City'] == 'kraków']
data = data[data['Currency'] == 'pln']
data = data[data['Property'] == 'flat']
data = data[(data['Amount'] >= data['Amount'].quantile(0.025))]
data = data[(data['Amount'] <= data['Amount'].quantile(0.975))]
data = data[(data['Area'] >= data['Area'].quantile(0.01))]
data = data[(data['Area'] <= data['Area'].quantile(0.99))]
data = data[data['District'] != 'unknown']
data = data[data['District'].isna() == False]
data = data[data['Seller'].isna() == False]
data = data[data['Description'].isna() == False]

In [236]:
data = data.reset_index(drop=True)

In [237]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3269 entries, 0 to 3268
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         3269 non-null   object 
 1   City         3269 non-null   object 
 2   District     3269 non-null   object 
 3   Amount       3269 non-null   float64
 4   Currency     3269 non-null   object 
 5   Property     3269 non-null   object 
 6   Seller       3269 non-null   object 
 7   Area         3269 non-null   float64
 8   Rooms        3221 non-null   float64
 9   Bathrooms    1576 non-null   float64
 10  Parking      3269 non-null   object 
 11  Garden       3269 non-null   bool   
 12  Balcony      3269 non-null   bool   
 13  Terrace      3269 non-null   bool   
 14  Floor        3269 non-null   bool   
 15  New          3269 non-null   bool   
 16  Estate       3269 non-null   bool   
 17  Townhouse    3269 non-null   bool   
 18  Apartment    3269 non-null   bool   
 19  Land  

### Impute missing values

The next step is to fill in missing values for numeric columns `Amount` `Area` `Rooms` and `Bathrooms`. We use the KNN algorithm to accomplish this.

In [238]:
numeric = list(data.select_dtypes('number').columns)

In [239]:
mask = (data['Bathrooms'].isna() == True | data['Rooms'].isna())
missing = data[numeric]

imputer = KNNImputer(n_neighbors=5)
imputer.fit(missing)

missing = imputer.transform(missing)
missing = pd.DataFrame(missing, columns=numeric)

for col in numeric:
    data[col] = missing[col]
    
for col in numeric:
    data[col] = data[col].apply(lambda x: round(x))    

In [240]:
print(data.shape)

(3269, 24)


## Save data

Verify that there are no `NaN`s in data.

In [241]:
data.isnull().sum().sum()

0

In [242]:
data = data.drop(['Title', 
                  'Description', 
                  'Link', 
                  'Property', 
                  'City', 
                  'Currency', 
                  'Date'], axis=1)

In [243]:
data.head()

,District,Amount,Seller,Area,Rooms,Bathrooms,Parking,Garden,Balcony,Terrace,Floor,New,Estate,Townhouse,Apartment,Land,Studio
0,dębniki,990000,realtor,93,4,2,street,False,False,False,False,False,False,False,False,False,False
1,prądnik biały,401430,realtor,41,2,1,garage,False,True,True,False,True,False,False,False,True,False
2,krowodrza,439000,realtor,29,1,1,garage,False,True,False,True,False,True,False,False,False,True
3,krowodrza,520000,realtor,55,3,1,street,False,False,False,False,False,False,False,False,False,False
4,bieżanów,287387,realtor,38,2,1,garage,False,True,False,False,False,False,False,False,False,False


In [244]:
data.to_csv('../flats-data/cleaned_data.csv', index=False)